In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from PositionwiseFFN import PositionwiseFFN
from DotProductAttention import DotProductAttention
from MultiHeadAttention import MultiHeadAttention

In [2]:
# Test PFFN
dmodel=64
inlen = 60
ffn = PositionwiseFFN(dmodel)
x = torch.rand(32, 10, 64)
assert ffn(x).shape == x.shape


In [3]:
# Test dot queries = torch.normal(0, 1, (2, 1, 2))
### Test code from d2l.ai
queries = torch.normal(0, 1, (2, 10, 2))
keys = torch.normal(0, 1, (2, 10, 2))
values = torch.normal(0, 1, (2, 10, 4))
valid_lens = torch.tensor([2, 6])

attention = DotProductAttention(dropout=0.5)
attention.eval()
assert attention(queries, keys, values).shape == (2, 10, 4)


In [4]:
# Test MHA
d_model, num_heads = 100, 5
attention = MultiHeadAttention(d_model, num_heads, 0.5)
batch_size = 2

X = torch.ones((batch_size, inlen, d_model))

assert attention(X, X, X, "None").shape == (batch_size, inlen, d_model)